In [ ]:
import os
import json
import requests

## Configuration

In [ ]:
dockstore_api_uri = "http://uads-test-dockstore-deploy-lb-1762603872.us-west-2.elb.amazonaws.com:9998/api"

In [ ]:
token = !cat ~/.dockstore/config  | grep token | awk '{print $2}'
token = token[0]

In [ ]:
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
}

## User Info

In [ ]:
request_url = f"{dockstore_api_uri}/users/user"

In [ ]:
response = requests.get(request_url, headers=headers)

In [ ]:
response.status_code

In [ ]:
user_id = response.json()['id']

## Application Variables

In [ ]:
organization = "unity-sds"
git_version_control = "github"
git_registry = "github.com"
repository_base = "sounder-sips-application"
descriptor_type = "CWL"

## Publish L1A Algorithm

In [ ]:
workflow_path = "/cwl/l1a_workflow.cwl"
workflow_name = "sounder_sips_l1a"

In [ ]:
params = {
    'workflowRegistry': git_version_control,
    'workflowPath': f"{organization}/{repository_base}",
    'defaultWorkflowPath': workflow_path,
    'workflowName': workflow_name,
    'descriptorType': descriptor_type,
    #'defaultTestParameterFilePath':, 
}

request_url = f"{dockstore_api_uri}/workflows/manualRegister"

In [ ]:
response = requests.post(request_url, params=params, headers=headers)

In [ ]:
response.status_code

In [ ]:
workflow_id = response.json()['id']

## Refresh

In [ ]:
request_url = f"{dockstore_api_uri}/workflows/{workflow_id}/refresh"

In [ ]:
response = requests.get(request_url, headers=headers)

In [ ]:
response.status_code

## Publish

In [ ]:
request_url = f"{dockstore_api_uri}/workflows/{workflow_id}/publish"

In [ ]:
data = { 'publish': True }

In [ ]:
response = requests.post(request_url, data=json.dumps(data), headers=headers)

In [ ]:
response.status_code

## List Algorithms

In [ ]:
request_url = f"{dockstore_api_uri}/users/{user_id}/workflows"

In [ ]:
response = requests.get(request_url, headers=headers)

In [ ]:
response.status_code

In [ ]:
for wf in response.json():
    print(f"{wf['id']} {wf['full_workflow_path']} - Published: {wf['is_published']}")

## Workflow ID from Repository Name

In [ ]:
request_repo = f"{git_registry}/{organization}/{repository_base}/{workflow_name}".replace("/", "%2F")
request_url = f"{dockstore_api_uri}/workflows/path/entry/{request_repo}"

In [ ]:
response = requests.get(request_url, headers=headers)

In [ ]:
response.status_code

In [ ]:
#response.json()

In [ ]:
workflow_id = response.json()['id']
workflow_id

## Unpublish

In [ ]:
request_url = f"{dockstore_api_uri}/workflows/{workflow_id}/publish"

In [ ]:
data = { 'publish': False }

In [ ]:
response = requests.post(request_url, data=json.dumps(data), headers=headers)

In [ ]:
response.status_code

## Restub

In [ ]:
request_url = f"{dockstore_api_uri}/workflows/{workflow_id}/restub"

In [ ]:
response = requests.get(request_url, headers=headers)

In [ ]:
response.status_code

## Delete

In [ ]:
request_repo = f"{repository_base}/{workflow_name}".replace("/", "%2F")
request_url = f"{dockstore_api_uri}/workflows/registries/{git_registry}/organizations/{organization}/repositories/{request_repo}"

In [ ]:
response = requests.delete(request_url, headers=headers)

In [ ]:
response.status_code

In [ ]:
response.content